In [1]:
#Imports das classes criadas e demais classes necessárias
from Classes.Arquivo import Arquivo
from Classes.BancoDDL import BancoDDL
from Classes.BancoDML import BancoDML
from Classes.BancoDQL import BancoDQL
from Classes.AnaliseGenero import AnaliseGenero
from Classes.AnaliseArmaFogo import AnaliseArmaFogo
from Classes.AnaliseRegiao import AnaliseRegiao
import plotly.express as px
import pandas as pd

In [2]:
#Carregando e inspecionando todos os CSVs
try:
    #Criação dos DataFrames
    s_total = Arquivo("Arquivos CSV/suicidios-total.csv")
    homens_arma = Arquivo("Arquivos CSV/suicidios-homens-armas-fogo.csv")
    mulheres_arma = Arquivo("Arquivos CSV/suicidios-mulheres-armas-fogo.csv")
    s_homens = Arquivo("Arquivos CSV/suicidios-homens.csv")
    s_mulheres = Arquivo("Arquivos CSV/suicidios-mulheres.csv")
    s_regiao = Arquivo("Arquivos CSV/suicidios-regiao.csv")
    verificador_geral = s_total.verifica and homens_arma.verifica and mulheres_arma.verifica and s_homens.verifica and s_mulheres.verifica and s_regiao.verifica

    #Inspeção
    if verificador_geral:
        print("Arquivos CSV inspecionados com sucesso!")
    else:
        print(f"""
Status de verificação de cada arquivo:
suicidios-total.csv = {s_total.verifica}
suicidios-homens-armas-fogo.csv = {homens_arma.verifica}
suicidios-mulheres-armas-fogo.csv = {mulheres_arma.verifica}
suicidios-homens.csv = {s_homens.verifica}
suicidios-mulheres.csv = {s_mulheres.verifica}
suicidios-regiao.csv = {s_regiao.verifica}
        """)
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivos CSV inspecionados com sucesso!


In [3]:
#Conexão localhost para realizar consultas DQL
BancoLocal = BancoDQL('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')

Conexão bem-sucedida!


In [ ]:
#Criando tabelas no banco local (localhost) PostgresSQL
escolha = input("Quer criar as tabelas no banco de dados local? (Sim/Não) \nResposta:")
match escolha.lower():
    case "sim":
            CriacaoBanco = BancoDDL('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')
            CriacaoBanco.criacao_tabelas()
            CriacaoBanco.encerrar()
    case "não":
        print("Inserção parada")
    case _:
        print("Escolha inválida")

In [4]:
#Instanciando DataFrames para usar na inserção dos dados
colunasRegiao = BancoLocal.buscar_dados('regiao')[['reg_cod', 'nome']]
regiaoDf = s_regiao.dataframe[['nome', 'período', 'valor']]
mulherFogoDf = mulheres_arma.dataframe[['período', 'valor']]
homemFogoDf = homens_arma.dataframe[['período', 'valor']]

In [ ]:
#Realizando inserção dos dados no banco de dados local

escolha = input("Quer inserir os valores em todas as tabelas? (Sim/Não) \nResposta:")
match escolha.lower():
    case "sim":
        BancoLocalDML = BancoDML('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')

        for table in range(4):
            match table:
                case 0:
                    #Table periodo:
                    BancoLocalDML.inserir_total(s_total.dataframe)
                    colunasPeriodo = BancoLocal.buscar_dados('periodo')[['per_cod', 'ano']]
                    print("Dados total inseridos")
                    
                case 1:
                    #Table gen_periodo:
                    BancoLocalDML.inserir_gen(s_homens.dataframe, s_mulheres.dataframe, colunasPeriodo)
                    print("Dados gênero inseridos")

                case 2:
                    #Table reg_periodo:
                    BancoLocalDML.inserir_reg(colunasRegiao, regiaoDf,  colunasPeriodo)
                    print("Dados região por período inseridos")

                case 3:
                    #Table arma_fogo:
                    BancoLocalDML.inserir_dados_fogo(mulherFogoDf, homemFogoDf, colunasPeriodo)
                    print("Dados armas de fogo inseridos")

        BancoLocalDML.encerrar()
        print("Sucesso!")
    case "não":
        print("Inserção parada")
    case _:
        print("Escolha inválida")

In [5]:
#Gráfico de setores Porcentagem de suicídios por gênero
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')
dadosSuicidiosGeral = s_total.dataframe['valor'].sum() - dadosGeneros['quantidade'].sum()

soma_generos = dadosGeneros.groupby(['gen_cod','descricao','tipo'], as_index=False).sum()
novo_dado = pd.DataFrame({'gen_cod': [3], 'quantidade': [dadosSuicidiosGeral], 'descricao': ['IGN']})
dados_final = pd.concat([soma_generos, novo_dado], ignore_index=True)

analiseGenero = AnaliseGenero(dados_final, 'quantidade', 'descricao', 'quantidade', 'pie')
analiseGenero.criar_grafico()

In [6]:
#Gráfico histograma Quantidade de suicídios por gênero em cada período
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')

dadosGeneros = dadosGeneros.rename(columns={'descricao':'Gêneros', 'quantidade':'Quantidade', 'ano':'Período'})

analiseGeneroPeriodo = AnaliseGenero(dadosGeneros, 'Período', 'Quantidade', 'Gêneros', 'histogram')
analiseGeneroPeriodo.criar_grafico()

In [ ]:
#Modelo de regressão polinomial
analiseGeneroPoli = AnaliseGenero(dadosGeneros, 'Quantidade', 'Gêneros', 'Quantidade', 'pie')
analiseGeneroPoli.modelo_regre_poli(grau_homens=2, grau_mulheres=2)

Modelo Polinomial - Homens
Previsão de suicídios (2023-2030): [12928.52597409 13540.49548282 14179.46287411 14845.42814795
 15538.39130432 16258.35234327 17005.31126475 17779.26806878]
Desvio padrão: 247.92
Desvio padrão em porcentagem: 2.96%

Modelo Polinomial - Mulheres
Previsão de suicídios (2023-2030): [3596.18181814 3777.2298136  3966.41473736 4163.73658941 4369.19536974
 4582.79107837 4804.52371528 5034.39328047]
Desvio padrão: 91.16
Desvio padrão em porcentagem: 4.03%


In [9]:
#Gráfico de linha Quantidade de suicídios por região em cada período 
dadosRegPeriodo = BancoLocal.buscar_dados('reg_periodo')

dadosRegPeriodo = dadosRegPeriodo.rename(columns={'quantidade':'Quantidade','ano':'Período','descricao':'Descrição'})

analiseRegiaoPeriodo = AnaliseRegiao(dadosRegPeriodo, 'Período', 'Quantidade', 'Descrição')
analiseRegiaoPeriodo.criar_grafico()
analiseRegiaoPeriodo.anos_max_min()

Resultados (Ano_Maior e Ano_Menor por Região nos últimos 10 anos):
      Descrição  Ano_Maior  Maior_Valor  Ano_Menor  Menor_Valor
0      Nordeste       2022         3837       2014         2393
1       Sudeste       2022         6022       2013         3959
2         Norte       2022         1312       2014          708
3  Centro-Oeste       2022         1594       2015          940
4           Sul       2022         3697       2014         2319


In [10]:
#Gráfico de bolhas Quantidade de suicídios por arma de fogo em cada período
dadosArmaFogo = BancoLocal.buscar_dados('arma_fogo')
df = px.data.gapminder()
dadosArmaFogo["size_scaled"] = (dadosArmaFogo["quantidade_total"] - dadosArmaFogo["quantidade_total"].min()) / \
                               (dadosArmaFogo["quantidade_total"].max() - dadosArmaFogo["quantidade_total"].min()) * 50
dadosArmaFogo = dadosArmaFogo.rename(columns={'ano':'Período', 'quantidade_total':'Quantidade'})
analiseArma = AnaliseArmaFogo(dadosArmaFogo, 'Período', 'Quantidade', 'Quantidade')
analiseArma.criar_grafico()

In [11]:
#Encerrando a conexão com o banco de dados 
BancoLocal.encerrar()